# ELO Merchant Category Recommendation

![](https://storage.googleapis.com/kaggle-competitions/kaggle/10445/logos/thumb76_76.png?t=2018-10-24-17-14-05)


Imagine being hungry in an unfamiliar part of town and getting restaurant recommendations served up, based on your personal preferences, at just the right moment. The recommendation comes with an attached discount from your credit card provider for a local place around the corner!

Right now, Elo, one of the largest payment brands in Brazil, has built partnerships with merchants in order to offer promotions or discounts to cardholders. But do these promotions work for either the consumer or the merchant? Do customers enjoy their experience? Do merchants see repeat business? Personalization is key.

Elo has built machine learning models to understand the most important aspects and preferences in their customers’ lifecycle, from food to shopping. But so far none of them is specifically tailored for an individual or profile. This is where you come in.

In this competition, Kagglers will develop algorithms to identify and serve the most relevant opportunities to individuals, by uncovering signal in customer loyalty. Your input will improve customers’ lives and help Elo reduce unwanted campaigns, to create the right experience for customers.


### imports

In [ ]:
import os
import gc
import time
import warnings
import datetime
import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns
from math import sqrt
import lightgbm as lgb
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold

print(os.listdir("../input"))
warnings.filterwarnings('ignore')

### Reduce Memory Usage do to high memory req.

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

### to be able to view all the columns

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

### generic missing percentage viewer for each column

In [ ]:
def missing_percentage(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
    missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data.head(20)

### loading the data...

In [ ]:
train        = pd.read_csv('../input/train.csv', parse_dates=["first_active_month"])
sample       = pd.read_csv('../input/sample_submission.csv')
test         = pd.read_csv('../input/test.csv', parse_dates=["first_active_month"])
ht           = pd.read_csv('../input/historical_transactions.csv',parse_dates=['purchase_date'])
merchant     = pd.read_csv('../input/merchants.csv')
new_merchant = pd.read_csv('../input/new_merchant_transactions.csv',parse_dates=["purchase_date"])

### reducing the memory for each file & overwriting

In [ ]:
train        = reduce_mem_usage(train)
test         = reduce_mem_usage(test)
ht           = reduce_mem_usage(ht)
new_merchant = reduce_mem_usage(new_merchant)

## EDA START

## Missing Value Exploration

In [ ]:
for df in [ht,new_merchant]:
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)

In [ ]:
feature_1 = train.loc[train['feature_1'] == 1]
feature_2 = train.loc[train['feature_1'] == 2]
feature_3 = train.loc[train['feature_1'] == 3]
feature_4 = train.loc[train['feature_1'] == 4]
feature_5 = train.loc[train['feature_1'] == 5]

plt.figure(figsize=(10, 6))
plt.title('Feature_2 Distribution based on Feature_1 values')
sns.despine()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

sns.distplot(feature_1['feature_2'], hist=False, rug=False,label='1')
sns.distplot(feature_2['feature_2'], hist=False, rug=False,label='2')
sns.distplot(feature_3['feature_2'], hist=False, rug=False,label='3')
sns.distplot(feature_4['feature_2'], hist=False, rug=False,label='4')
sns.distplot(feature_5['feature_2'], hist=False, rug=False,label='5')

In [ ]:
feature_1 = train.loc[train['feature_2'] == 1]
feature_2 = train.loc[train['feature_2'] == 2]
feature_3 = train.loc[train['feature_2'] == 3]

plt.figure(figsize=(10, 6))
plt.title('Feature_1 Distribution based on Feature_2 values')
sns.despine()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

sns.distplot(feature_1['feature_1'], hist=False, rug=False,label='1')
sns.distplot(feature_2['feature_1'], hist=False, rug=False,label='2')
sns.distplot(feature_3['feature_1'], hist=False, rug=False,label='3')

In [ ]:
feature_1 = train.loc[train['feature_3'] == 0]
feature_2 = train.loc[train['feature_3'] == 1]

plt.figure(figsize=(10, 6))
plt.title('Feature_1 Distribution based on Feature_3 values')
sns.despine()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

sns.distplot(feature_1['feature_1'], hist=False, rug=False,label='0')
sns.distplot(feature_2['feature_1'], hist=False, rug=False,label='1')

In [ ]:
feature_1 = train.loc[train['feature_3'] == 0]
feature_2 = train.loc[train['feature_3'] == 1]

plt.figure(figsize=(10, 6))
plt.title('Feature_2 Distribution based on Feature_3 values')
sns.despine()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

sns.distplot(feature_1['feature_2'], hist=False, rug=False,label='0')
sns.distplot(feature_2['feature_2'], hist=False, rug=False,label='1')

In [ ]:
# thanks to this kernel @ https://www.kaggle.com/artgor/elo-eda-and-models
fig, ax = plt.subplots(3, 1, figsize = (12, 12))
train['feature_1'].value_counts().sort_index().plot(kind='bar', ax=ax[0], color='teal')
train['feature_2'].value_counts().sort_index().plot(kind='bar', ax=ax[1], color='brown')
train['feature_3'].value_counts().sort_index().plot(kind='bar', ax=ax[2], color='gold');

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('Target Distribution')
sns.despine()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

sns.distplot(train['target'], hist=True, rug=False,norm_hist=True)

## EDA START

## categorical modifications + aggregations

In [ ]:
categorical_feats = ['feature_1', 'feature_2', 'feature_3']

for col in categorical_feats:
    
    lbl = LabelEncoder()
    lbl.fit(list(train[col].values.astype('str')) + list(test[col].values.astype('str')))
    train[col] = lbl.transform(list(train[col].values.astype('str')))
    test[col]  = lbl.transform(list(test[col].values.astype('str')))
    
df_all = pd.concat([train, test])
df_all = pd.get_dummies(df_all, columns=categorical_feats)

len_train = train.shape[0]

train = df_all[:len_train]
test  = df_all[len_train:]

In [ ]:
ht['authorized_flag'] = ht['authorized_flag'].map({'Y':1, 'N':0})
ht['category_1']      = ht['category_1'].map({'Y': 1, 'N': 0})
ht['category_2x1']    = (ht['category_2'] == 1) + 0
ht['category_2x2']    = (ht['category_2'] == 2) + 0
ht['category_2x3']    = (ht['category_2'] == 3) + 0
ht['category_2x4']    = (ht['category_2'] == 4) + 0
ht['category_2x5']    = (ht['category_2'] == 5) + 0
ht['category_3A']     = (ht['category_3'].astype(str) == 'A') + 0
ht['category_3B']     = (ht['category_3'].astype(str) == 'B') + 0
ht['category_3C']     = (ht['category_3'].astype(str) == 'C') + 0

ht['purchase_date'] = pd.to_datetime(ht['purchase_date'])
ht['year']          = ht['purchase_date'].dt.year
ht['weekofyear']    = ht['purchase_date'].dt.weekofyear
ht['month']         = ht['purchase_date'].dt.month
ht['dayofweek']     = ht['purchase_date'].dt.dayofweek
ht['weekend']       = (ht.purchase_date.dt.weekday >=5).astype(int)
ht['hour']          = ht['purchase_date'].dt.hour


ht['Christmas_Day_2017'] = (pd.to_datetime('2017-12-25') - ht['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
ht['fathers_day_2017']   = (pd.to_datetime('2017-08-13') - ht['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
ht['Children_day_2017']  = (pd.to_datetime('2017-10-12') - ht['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
ht['Black_Friday_2017']  = (pd.to_datetime('2017-11-24') - ht['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
ht['Valentine_day_2017'] = (pd.to_datetime('2017-06-12') - ht['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
ht['Mothers_Day_2018']   = (pd.to_datetime('2018-05-13') - ht['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)

def aggregate_historical_ht(history):
    
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).\
                                      astype(np.int64) * 1e-9
    
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'category_1': ['sum', 'mean'],
        'category_2': ['nunique'],
        'category_3A': ['sum'],
        'category_3B': ['sum'],
        'category_3C': ['sum'],
        'category_2x1': ['sum','mean'],
        'category_2x2': ['sum','mean'],
        'category_2x3': ['sum','mean'],
        'category_2x4': ['sum','mean'],
        'category_2x5': ['sum','mean'],   
        'merchant_id': ['nunique'],
        'merchant_category_id': ['nunique'],
        'city_id': ['nunique'],
        'purchase_amount': ['sum', 'median', 'max', 'min', 'std'],
        'installments': ['sum', 'median', 'max', 'min', 'std'],
        'purchase_date': [np.ptp, 'max', 'min'],
        'weekofyear': ['nunique'],
        'weekend' : ['sum', 'mean'],
        'dayofweek': ['nunique'],
        'hour': ['nunique'],
        'year': ['nunique'],
        'card_id' : ['size'],
        'month': ['nunique'],
        'state_id': ['nunique'],
        'subsector_id': ['nunique'],
        'Christmas_Day_2017':['mean'],
        'fathers_day_2017':['mean'],
        'Children_day_2017':['mean'],
        'Black_Friday_2017':['mean'],
        'Valentine_day_2017':['mean'],
        'Mothers_Day_2018':['mean']
        }
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['hist_' + '_'.join(col).strip() 
                           for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='hist_ht_count'))
    
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
    
    return agg_history

history = aggregate_historical_ht(ht)
del ht
gc.collect()

In [ ]:
train = pd.merge(train, history, on='card_id', how='left')
test  = pd.merge(test, history, on='card_id', how='left')

### Feature Generation for time related feautures

In [ ]:
# thanks to the kernel @ https://www.kaggle.com/yhn112/data-exploration-lightgbm-catboost-lb-3-760

for df in [train,test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['year'] = df['first_active_month'].dt.year
    df['month'] = df['first_active_month'].dt.month
    df['week'] = df['first_active_month'].dt.week
    df['elapsed_time'] = (datetime.date(2018, 2, 1) - df['first_active_month'].dt.date).dt.days

train.drop('first_active_month',axis=1,inplace=True)
test.drop('first_active_month',axis=1,inplace=True)

### more aggregations

In [ ]:
new_merchant['purchase_date'] = pd.DatetimeIndex(new_merchant['purchase_date']).\
                                astype(np.int64) * 1e-9

new_merchant['authorized_flag'] = new_merchant['authorized_flag'].map({'Y':1, 'N':0})
new_merchant['category_1']      = new_merchant['category_1'].map({'Y':1, 'N':0})
new_merchant['category_3A']     = (new_merchant['category_3'].astype(str) == 'A') + 0
new_merchant['category_3B']     = (new_merchant['category_3'].astype(str) == 'B') + 0
new_merchant['category_3C']     = (new_merchant['category_3'].astype(str) == 'C') + 0
new_merchant['category_2x1']    = (new_merchant['category_2'] == 1) + 0
new_merchant['category_2x2']    = (new_merchant['category_2'] == 2) + 0
new_merchant['category_2x3']    = (new_merchant['category_2'] == 3) + 0
new_merchant['category_2x4']    = (new_merchant['category_2'] == 4) + 0
new_merchant['category_2x5']    = (new_merchant['category_2'] == 5) + 0

def aggregate_new_transactions(new_trans):    
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'category_1':   ['sum', 'mean'],
        'category_2':   ['nunique'],
        'category_3A':  ['sum'],
        'category_3B':  ['sum'],
        'category_3C':  ['sum'],     
        'category_2x1': ['sum','mean'],
        'category_2x2': ['sum','mean'],
        'category_2x3': ['sum','mean'],
        'category_2x4': ['sum','mean'],
        'category_2x5': ['sum','mean'],  
        'merchant_id': ['nunique'],
        'merchant_category_id': ['nunique'],
        'city_id': ['nunique'],
        'purchase_amount': ['sum', 'median', 'max', 'min', 'std'],
        'purchase_date': [np.ptp, 'max', 'min'],
        'installments': ['sum', 'median', 'max', 'min', 'std'],
        'month_lag': ['min', 'max'],
        'state_id': ['nunique'],
        'subsector_id': ['nunique']
        }
    agg_new_trans = new_trans.groupby(['card_id']).agg(agg_func)
    agg_new_trans.columns = ['new_' + '_'.join(col).strip() 
                           for col in agg_new_trans.columns.values]
    agg_new_trans.reset_index(inplace=True)
    
    df = (new_trans.groupby('card_id')
          .size()
          .reset_index(name='new_transactions_count'))
    
    agg_new_trans = pd.merge(df, agg_new_trans, on='card_id', how='left')
    
    return agg_new_trans

new_trans = aggregate_new_transactions(new_merchant)

del new_merchant
gc.collect()

In [ ]:
train = pd.merge(train, new_trans, on='card_id', how='left')
test  = pd.merge(test, new_trans, on='card_id', how='left')

### Reducing again after all these aggregations

In [ ]:
train = reduce_mem_usage(train)
test  = reduce_mem_usage(test)

In [ ]:
y     = train['target']
train = train.drop(['target'],axis=1)
test  = test.drop(['target'],axis=1)

# Light GBM (Model 1)

In [ ]:
id_train = train['card_id'].copy()
id_test  = test['card_id'].copy()

train = train.drop('card_id', axis = 1)
test  = test.drop('card_id', axis = 1)

nfolds = 10
folds = KFold(n_splits=5, shuffle=True, random_state=4590)

### Parameters

In [ ]:
param = {'num_leaves': 129,
         'min_data_in_leaf': 148, 
         'objective':'regression',
         'max_depth': 9,
         'learning_rate': 0.005,
         "min_child_samples": 24,
         "boosting": "gbdt",
         "feature_fraction": 0.7202,
         "bagging_freq": 1,
         "bagging_fraction": 0.8125 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.3468,
         "verbosity": -1}

### Feature Importance

In [ ]:
feature_importance_df = np.zeros((train.shape[1], nfolds))
mvalid = np.zeros(len(train))
mfull  = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, train.values)):
    print('----')
    print("fold n°{}".format(fold_))
    
    x0,y0 = train.iloc[trn_idx], y[trn_idx]
    x1,y1 = train.iloc[val_idx], y[val_idx]
    
    trn_data = lgb.Dataset(x0, label= y0); val_data = lgb.Dataset(x1, label= y1)
    
    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], 
                    verbose_eval=500, early_stopping_rounds = 150)
    mvalid[val_idx] = clf.predict(x1, num_iteration=clf.best_iteration)
    
    feature_importance_df[:, fold_] = clf.feature_importance()
    
    mfull += clf.predict(test, num_iteration=clf.best_iteration) / folds.n_splits
    
np.sqrt(mean_squared_error(mvalid, y))

In [ ]:
ximp = pd.DataFrame()
ximp['feature'] = train.columns
ximp['importance'] = feature_importance_df.mean(axis = 1)

plt.figure(figsize=(14,14))
sns.barplot(x="importance",
            y="feature",
            data=ximp.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()

 ### to check the distribution 

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('Target Distribution')
sns.despine()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.distplot(mfull,hist=True, rug=True,norm_hist=True)

In [ ]:
submission = pd.DataFrame()
submission['card_id']  = id_test
submission['target'] = mfull
submission.to_csv('submission_lgb.csv', index = False)

# *to be continued*
### different models & blending will be added soon